In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

In [ ]:
fullDF = pd.read_csv('/home/welling/git/synecoace/data/nsch_2016_topical.csv')
print fullDF.columns

## The Model ##

Consider two sequential ages, say 6 and 7 years.  Given an child at one age, the goal is to produce a *guide function* which can be used to select a collection of children at the other age which as a group describe the most likely match to the original child.  Specifically, given a sample of individuals at one age, the guide function should generate samples at the other age having the highest possible *mutual information* with the first group.

The guided samples will be selected by applying *Metropolis sampling* using the guide function.  This requires only that the guide function be positive definite and integrable.  In essense the guide function is proportional to the likelihood that a sampled child at one age matches a specific child at the other age.

We will develop an appropriate guide function by defining a function of an appropriate form in terms of a set of weights.  Samples will be drawn based on the given weights, the mutual information will be calculated, and the weights will be updated so as to increase the mutual information.  This process will be repeated until the mutual information is maximized.  The set of weights which produces the maximum mutual information is the optimal choice of weights given the initial choice of form for the guide function.

## Factors we are using ##
* ACE7: experienced violence
* ACE8: drugs and alcohol
* SC_AGE_YEARS: selected child age
* K4Q32X01: vision tested by eye doctor (2.0 -> False)
* K7Q30: sports teams
* K7Q31: clubs or organizations
* FPL: percent of federal poverty level?
* AGEPOS4: birth order
* FWC: sample weight
* YEAR: survey year

In [ ]:
subDF=fullDF[['ACE7', 'ACE8', 'FWC', 'YEAR', 'FPL', 'SC_AGE_YEARS','K4Q32X01', 'K7Q30', 'K7Q31', 'AGEPOS4']]
subDF = subDF.dropna()
print len(subDF)
#print subDF

In [ ]:
age6DF = subDF[subDF.SC_AGE_YEARS==6]
print len(age6DF)
age7DF = subDF[subDF.SC_AGE_YEARS==7]
print len(age7DF)

Select samples from the table in a weighted fashion

In [ ]:
def mkSamps(df, nSamp):
    fracWt = df['FWC']/df['FWC'].sum()
    choices = np.random.choice(len(df), nSamp, p=fracWt)
    return df.iloc[choices].drop(columns=['FWC'])
nSamp = 10
print mkSamps(age6DF, nSamp)


In [ ]:
COLUMN_DICT = {key : idx for idx, key in enumerate(mkSamps(subDF, 1).columns)}
print COLUMN_DICT

FPL needs to be binned to form a histogram to get the quantization we need for this.  Bins of 50 seems OK.

In [ ]:
fplSamps = mkSamps(subDF, 100000)['FPL']
plt.hist(fplSamps, bins=100)
plt.title('Histogram of FPL values')
plt.show()

We need to enumerate the possible states for our variables.  Assume fixed age and ignore the ACE components for the moment.  

In [ ]:
def whichBin(sampV):
    """
    Input is an ndarray of sample values
    """
    fplBinWidth = 50
    fplMin = 50
    bin = np.abs((sampV[:, COLUMN_DICT['FPL']] - fplMin) // 50).astype('int')
    assert (bin >= 0).all() and (bin < 8).all(), 'FPL out of range?'
    nBins = 8
    # Each of the following is either 1.0 or 2.0
    bin = 2 * bin + (sampV[:, COLUMN_DICT['K4Q32X01']] == 1.0)
    nBins *= 2
    bin = 2 * bin + (sampV[:, COLUMN_DICT['K7Q30']] == 1.0)
    nBins *= 2
    bin = 2 * bin + (sampV[:, COLUMN_DICT['K7Q31']] == 1.0)
    nBins *= 2
    return bin, nBins

def scatter(idx, vals, target):
    """target[idx] += vals, but allowing for repeats in idx"""
    np.add.at(target, idx.ravel(), vals.ravel())


In [ ]:
def toHisto(sampV):
    """Generate a histogram of sample bins"""
    binV, nBins = whichBin(sampV)
    targ = np.zeros([nBins], dtype=np.int32)
    vals = np.ones([len(sampV)], dtype=np.int32)
    scatter(binV, vals, targ)
    return targ

In [ ]:
def toProbV(sampV):
    sampH = toHisto(sampV)
    probV = sampH.astype(np.float64)
    probV /= np.sum(probV)
    return probV

In [ ]:
sampV = mkSamps(subDF, 1000)
print sampV.columns
print COLUMN_DICT['FPL']
sampH = toHisto(sampV.values)
print sampH
probV = toProbV(sampV.values)
print probV

Given the probV we can calculate mutual information:
$$
I(X;Y) = \sum_{y \in Y} \sum_{x \in X} p(x, y) log \left (\frac{p(x,y)}{p(x)p(y)} \right )
$$

In [ ]:
def mutualInfo(sampVX, sampVY):
    assert len(sampVX) == len(sampVY), 'Sample vector lengths do not match'
    binVX, nBinsX = whichBin(sampVX)
    binVY, nBinsY = whichBin(sampVY)
    assert nBinsX == nBinsY, 'Unexpectedly got different bin counts?'
    cA = np.zeros([nBinsX, nBinsX], dtype=np.int32)
    idxV = np.ravel_multi_index(np.array([binVX, binVY]), (nBinsX, nBinsX))
    np.add.at(cA.ravel(), idxV, np.ones(len(idxV), dtype=np.int32).ravel())
    pA = cA.astype(np.float32)
    pA /= sum(pA.ravel())
    xPV = toProbV(sampVX)
    yPV = toProbV(sampVY)
    xyPA = np.einsum('i,j->ij', xPV, yPV)  # einsum is my new favorite function
    oldErr = np.seterr(invalid='ignore', divide='ignore')
    prodA = pA * np.nan_to_num(np.log(pA / xyPA))  # element-wise calculation
    np.seterr(**oldErr)
    return np.sum(prodA.ravel())

sampX = mkSamps(subDF, 30000)
sampY = mkSamps(subDF, 30000)
print mutualInfo(sampX.values, sampY.values)

Define a scalar function on a pair of samples and a weight vector

In [ ]:
wtSer = pd.Series({'YEAR': 1.0,
                   'FPL':1.0e-4,  # because it's percent
                   'SC_AGE_YEARS': 1.0, 
                   'K4Q01': 1.0, 
                   'K4Q32X01': 1.0,
                   'K7Q30':1.0,
                   'K7Q31': 1.0,
                   'AGEPOS4': 1.0}, index=subDF.columns)
wtSer = wtSer.drop(labels=['ACE7', 'ACE8', 'FWC'])  # get the right index order but no extra entries
print wtSer
print wtSer.values
print wtSer.index

In [ ]:
def funV(samps1V, samps2V, wtSerV):
    """
    Returns a numpy vector the columns of which correspond to the input samples
    """
    wtA = wtSerV
    #print wtA.shape
    #print samps1V.shape
    offset = samps1V.shape[1] - wtSerV.shape[0]
    #print offset
    samp1A = samps1V[:, offset:]
    samp2A = samps2V[:, offset:]
    delta = samp1A - samp2A
    print 'delta', delta
    delta *= delta
    print 'delta^2', delta
    return np.exp(-np.asmatrix(wtA) * np.asmatrix(delta).transpose())
    

In [ ]:
def lnLik(samps1V, samps2V, wtSerV):
    """
    funV has the right shape to fill the role of likelihood in the Metropolis algorithm.  We'll
    take the log, and use it as a log likelihood.
    """
    wtA = wtSerV
    offset = samps1V.shape[1] - wtSer.shape[0]
    samp1A = samps1V[:, offset:]
    samp2A = samps2V[:, offset:]
    delta = samp1A - samp2A
    delta *= delta
    return np.asarray((-np.asmatrix(wtA) * np.asmatrix(delta).transpose())).reshape((-1, 1))
    

In [ ]:
print funV(mkSamps(subDF, 3).values, mkSamps(subDF, 3).values, wtSer)
llk = lnLik(mkSamps(subDF, 3).values, mkSamps(subDF, 3).values, wtSer)
print llk


In [ ]:
# Now we need a mutator
def mutate(sampV, df, stepSzV):
    """
    Return a 'mutated' version of sampV, based on the given step sizes.  Unfortunately our samples
    are discrete and come from a table, so I'm not sure how to do this unless we first generate
    a proximity network of some sort, so for the moment let's just generate a new set of samples-
    this corresponds to an infinitely wide mutator.
    """
    return mkSamps(df, len(sampV)).values

In [ ]:
# And now we can write the Metropolis code.
# Thanks to https://python4mpia.github.io/fitting_data/Metropolis-Hastings.html
# initial guess for alpha as array.

nSamp = 100
testSamps = mkSamps(age6DF, nSamp)
guess = mkSamps(age7DF, nSamp)
# Prepare storing MCMC chain as array of arrays.
A = [guess.values]
# define stepsize of MCMC.
stepsizes = np.empty([nSamp])
stepsizes.fill(0.005)
nIter = 10000

In [ ]:
# Metropolis-Hastings with nIter iterations.
accepted  = np.zeros([nSamp, 1], dtype=np.int)
onesV = np.ones([nSamp], dtype=np.int).reshape((-1, 1))
zerosV = np.zeros([nSamp], dtype=np.int).reshape((-1, 1))
for n in range(nIter):
    oldAlpha  = A[-1]  # old parameter value as array
    oldLnLik = lnLik(testSamps.values, oldAlpha, wtSer)
    newAlpha = mutate(oldAlpha, age7DF, stepsizes)
    newLnLik = lnLik(testSamps.values, newAlpha, wtSer)
    if (n % 100 == 0):
        print '%s: %s' % (n, mutualInfo(testSamps.values, newAlpha))
    choices = np.logical_or(newLnLik > oldLnLik,
                            np.random.random(newLnLik.shape) < np.exp(newLnLik - oldLnLik))
    rslt = np.choose(choices, [oldAlpha, newAlpha])
    A.append(rslt)
    accepted += np.choose(choices, [zerosV, onesV])

plt.hist(accepted)
acceptanceRate = accepted/float(nIter)
plt.show()
plt.hist(acceptanceRate)
plt.show()

In [ ]:
nKeep = accepted.min()/10  # 10 mutations between samples to avoid correlation
keepStep = nIter//nKeep
burnIn = 10 * keepStep
print nKeep, keepStep, burnIn
assert burnIn < nIter, 'Not enough iterations for burn-in'
clean = []
for idx, sV in enumerate(A[burnIn:]):
    if idx % keepStep == 0:
        clean.append(sV)


In [ ]:
cleanV = np.concatenate(clean)
print cleanV[-4:-1,...]
print cleanV.shape
print age7DF.columns
print testSamps.values[-1, ...]
print len(testSamps)
print len(clean)

In [ ]:
expandedTestV = np.repeat(testSamps.values, len(clean), axis=0)
print mutualInfo(expandedTestV, cleanV)
print mutualInfo(expandedTestV, mkSamps(age7DF, cleanV.shape[0]).values)
print mutualInfo(expandedTestV, expandedTestV)
print mutualInfo(cleanV, cleanV)

## Ignore the rest of this ##
These bits are just left-overs

## Ignore the rest of this ##
These bits are just left-overs

In [ ]:
wtDF = pd.DataFrame({'FPL':[1.0, 1.0], 'SC_AGE_YEARS': [1.0, 1.0], 'K4Q01': [1.0, 1.0], 'K4Q32X01': [1.0, 1.0],
       'K7Q30':[1.0, 1.0], 'K7Q31': [1.0, 1.0], 'AGEPOS4': [1.0, 1.0]}, index=['ACE7', 'ACE8'])
print wtDF

In [ ]:
def funV(samps1, samps2, wtDF):
    """
    Returns a numpy matrix the rows of which correspond to ACE7 and ACE8 and
    the columns of which correspond to the input samples
    """
    wtA = wtDF.values
    offset = samps1.values.shape[1] - wtDF.values.shape[1]
    samp1A = samps1.values[:, offset:]
    samp2A = samps2.values[:, offset:]
    delta = samp1A - samp2A
    return np.asmatrix(wtA) * np.asmatrix(delta).transpose()
    

In [ ]:
print funV(mkSamps(subDF, 3), mkSamps(subDF, 3), wtDF)

$\mathcal L(\{M_1,M_2,\ldots,M_N\};\alpha) = \prod_{n=1}^N p(M_n|\alpha) = \prod_{n=1}^N c\left(\frac{M_n}{M_\odot}\right)^{-\alpha}$